In [2]:
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine
import yaml
from llama_index import SQLDatabase
from llama_index.llms import OpenAI
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

llm = OpenAI(model="gpt-4")

from llama_index.prompts import PromptTemplate

Dans ce premier prompt vous devez definir plusieurs choses:
- Comment le LLM doit se comporter (1)
- Quelles colonnes il doit obligatoirement utiliser pour faire son analyse (2)
- Description de la table pour aider le LLM (3)
- Guider le LLM pour sa query (exemple avec le pays) (4)
- Donner la bonne syntaxe (MSSQL vs MYSQL vs PostgreSQL) (5)
- Template natif à Llamaindex (6)

In [ ]:
text_to_sql_tmpl = """\
Given an input question, first create a syntactically correct {dialect} \
query to run, then look at the results of the query and return the answer. \
You can order the results by a relevant column to return the most \
interesting examples in the database. Always select this relevant column in your query in order to write an analysis afterward \
Always select the BPI and ConsumerDemandGap in order to retrieve it in your analysis at the end.\
If the user specify the product type/company/product name, you will just at the end do an analysis but do not search for the product name/type in the table\

Pay attention to use only the column names that you can see in the schema \
description. Be careful to not query for columns that do not exist. \
Pay attention to which column is in which table. Also, qualify column names \
with the table name when needed. 


The table View_CombinedAnalysis contains 7 columns: [idbrand] "\
[brand] \
[id_country] \
[BPI] \
[ConsumerDemandGap] \
[OwnAvg] \
[PurchaseAvg] \
For the country: \

Select the 2 first letters of the ISO code in a list of the countries mentioned by the user in following format for id_country: \
For example: ['FR','GB','US'] for France, United Kingdom and the United States \

Pay attention to execute the queries in MSSQL language. Meaning that for example you will note execute a query using "LIMIT n" \
But rather using "TOP n" \
(example below are what you should execute) \
Example:
The good one:
SELECT TOP 1 brand,BPI,ConsumerDemandGap FROM View_CombinedAnalysis ORDER BY BPI DESC; 
The bad one:
SELECT brand FROM View_CombinedAnalysis ORDER BY BPI DESC LIMIT 1; 

You are required to use the following format, \
each taking one line:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use tables listed below.
{schema}


Question: {query_str}
SQLQuery: \
"""

Dans ce second prompt vous devez definir plusieurs choses:
- Quel rôle doit jouer le LLM (1)
- Quelle type d'analyse je souhaite faire  (2)
- Etant donné sur une question, la manière dont je synthétise ma réponse (3)
- Plus de détail (4)
- Template natif à Llamaindex (5)

In [ ]:
RESPONSE_SYNTHESIS_PROMPT_TMPL_V2 = (
    "As an expert in market trends and brand popularity, you have to provide complete and professional analysis of your result.\n"
    "For each brand you will provide me with a in-depth analysis on why is should be interesting to introduce the market with it.\n"
    "Given an input question, synthesize a response from the query results and analyze the BPI and the ConsumerDemandGap for each brands. Write this BPI.\n"
    "At the end you will add a line with (Source: BrandTrends Entertainment)\n"
    "Query: {query_str}\n"
    "SQL: {sql_query}\n"
    "SQL Response: {context_str}\n"
    "Response: "
)
text_to_sql_prompt = PromptTemplate(text_to_sql_tmpl)
response_synthesis_prompt = PromptTemplate(RESPONSE_SYNTHESIS_PROMPT_TMPL_V2)


Next step:
- Créer votre DB (Azure MSSQL est recommandable pour son partenariat avec Azure)
- Donner la configuration avec sql alchemy
- Créer la DB avec llamaindex

In [ ]:
try:
    # Read database configuration
    with open('config/config.yaml', 'r') as file:
        config = yaml.safe_load(file).get('database_credentials')
    # Create connection URL
    connect_url = URL.create(
        'mssql+pyodbc',
        username=config['username'],
        password=config['password'],
        host=config['host'],
        port=config['port'],
        database=config['database'],
        query=dict(driver='/opt/homebrew/Cellar/msodbcsql18/18.3.2.1/lib/libmsodbcsql.18.dylib')
    )

    # Create an engine
    engine = create_engine(connect_url)
except Exception as e:
    print(f"An error occurred: {e}")

finally:
    engine.dispose()

sql_database = SQLDatabase(engine,
                           include_tables=["View_CombinedAnalysis"],
                           view_support = True,)
print(sql_database)

Last step:
- Création du query engine, avec les prompts et db 
- Puis query en language naturel

In [ ]:
sql_query_engine = NLSQLTableQueryEngine(
    text_to_sql_prompt=text_to_sql_prompt,
    sql_database=sql_database,
    tables=["View_CombinedAnalysis"],
    response_synthesis_prompt=response_synthesis_prompt
)


In [ ]:
response = sql_query_engine.query("Present a shortlist of 3 to 5 brands suitable for licensing for a personal care company in United Kingdom. ")


print(str(response))